In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt


#tf.compat.v1.enable_eager_execution()
#tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

# Weights

In [4]:
weights = {
    # 5x5 conv, 1 input, 6 outputs
    'weights_conv_1': tf.Variable(tf.random.normal([5, 5, 1, 6])),
    # 5x5 conv, 6 inputs, 16 outputs
    'weights_conv_2': tf.Variable(tf.random.normal([5, 5, 6, 16])),
    #5x5 conv as in paper, 16 inputs, 120 outputs
    'weights_conv_3': tf.Variable(tf.random.normal([1, 1, 16, 120])),
    # fully connected, 5*5*16 inputs, 120 outputs
    'weights_dense_1': tf.Variable(tf.random.normal([5*5*16, 120])),
    # fully connected, 120 inputs, 84 outputs
    'weights_dense_2': tf.Variable(tf.random.normal([120, 84])),
    # 84 inputs, 10 outputs (class prediction)
    'weights_dense_3': tf.Variable(tf.random.normal([84, 10])),
}

masks = {
    # 5x5 conv, 1 input, 6 outputs
    'mask_conv_1': tf.Variable(tf.ones([5, 5, 1, 6]), trainable=False),
    # 5x5 conv, 6 inputs, 16 outputs
    'mask_conv_2': tf.Variable(tf.ones([5, 5, 6, 16]), trainable=False),
    #5x5 conv as in paper, 16 inputs, 120 outputs
    'mask_conv_3': tf.Variable(tf.ones([1, 1, 16, 120]), trainable=False),
    # fully connected, 5*5*16 inputs, 120 outputs
    'mask_dense_1': tf.Variable(tf.ones([5*5*120, 120]), trainable=False),
    # fully connected, 120 inputs, 84 outputs
    'mask_dense_2': tf.Variable(tf.ones([120, 84]), trainable=False),
    # 84 inputs, 10 outputs (class prediction)
    'mask_dense_3': tf.Variable(tf.ones([84, 10]), trainable=False),
}

biases = {
    #output depth
    'bias_conv_1': tf.Variable(tf.random.normal([6])),
    'bias_conv_2': tf.Variable(tf.random.normal([16])),
    'bias_dense_1': tf.Variable(tf.random.normal([120])),
    'bias_dense_2': tf.Variable(tf.random.normal([84])),
    'bias_dense_3': tf.Variable(tf.random.normal([10])),
}



# Wrappers

In [5]:
#conv2D with bias and relu activation

class CustomConvLayer(layers.Layer):

    def __init__(self, weights, mask, biases, strides, padding='SAME'):
        
        super(CustomConvLayer, self).__init__()
        self.w = weights
        self.m = mask
        self.b = biases
        self.s = strides
        self.p = padding

        
    def call(self, inputs):
        print('inputs',inputs)
        #print('weights', self.w)
        #print('masks', self.m)
        print('weights * masks',tf.multiply(self.w, self.m))
        x = tf.nn.conv2d(inputs, tf.multiply(self.w, self.m), strides=[1, self.s, self.s, 1], padding=self.p, data_format='NCHW')
        print('x', x)
        #print('bias', self.b)
        x = tf.nn.bias_add(x, self.b, 'NC...')
        #print('x', x)
        return tf.nn.tanh(x)
        

#Average Pooling Layer
class CustomPoolLayer(layers.Layer):
    
    def __init__(self, k=2, padding='valid'):#padding='VALID'):
        super(CustomPoolLayer, self).__init__()
        self.k = k
        self.p = padding
    
    def call(self, inputs):
#        return tf.keras.layers.AveragePooling2D(pool_size=(self.k, self.k), strides=None, padding=self.p, data_format='channels_first')(inputs)
        return tf.nn.avg_pool2d(inputs, ksize=[1, self.k, self.k,1], strides=[1, self.k, self.k, 1], padding=self.p, )#data_format='NCHW')
    
#Dense Layer with Bias
class CustomDenseLayer(layers.Layer):
    
    def __init__(self, weights, mask, bias, activation = 'tanh'):
        super(CustomDenseLayer, self).__init__()
        self.w = weights
        self.b = bias
        self.a = activation
        self.m = mask
        
    def call(self, inputs):
        #print('dense w',self.w)
        #print('dense i',inputs)
        x = tf.matmul(inputs, tf.multiply(self.w, self.m))
        #print('dense x',x)
        x = tf.nn.bias_add(x, self.b)
        if self.a == 'tanh':
            return tf.nn.tanh(x)
        if self.a == 'softmax':
            return tf.nn.softmax(x)


# Create Model

In [6]:
class CustomConvModel(tf.keras.Model):
    def __init__(self):
        super(CustomConvModel, self).__init__()
        self.conv1 = CustomConvLayer(weights['weights_conv_1'], masks['mask_conv_1'], biases['bias_conv_1'], 1, 'SAME')#'VALID')
        self.maxpool1 = CustomPoolLayer(k=2, padding='SAME')
        self.conv2 = CustomConvLayer(weights['weights_conv_2'], masks['mask_conv_2'], biases['bias_conv_2'], 1, 'VALID')
        self.maxpool2 = CustomPoolLayer(k=2, padding='VALID')
        self.conv3 = CustomConvLayer(weights['weights_conv_3'], masks['mask_conv_3'], biases['bias_dense_1'], 1, 'VALID')
        #self.dense1 = CustomDenseLayer(weights['weights_dense_1'], biases['bias_dense_1'], 'tanh')
        self.dense2 = CustomDenseLayer(weights['weights_dense_2'], masks['mask_dense_2'], biases['bias_dense_2'], 'tanh')
        self.dense3 = CustomDenseLayer(weights['weights_dense_3'], masks['mask_dense_3'], biases['bias_dense_3'], 'softmax')
        
    def call(self, inputs):
        x = tf.reshape(inputs, shape=[-1, 1, 28, 28])
        #print(x.shape)
        x = self.conv1(x)
        print('after conv1', x.shape)
        x = self.maxpool1(x)
        print('after pool1',x.shape)
        x = self.conv2(x)
        #print('after conv2',x.shape)
        x = self.maxpool2(x)
        #print('yo',x.shape)
        #x = layers.Flatten()(x)
        print('after pool2',x.shape)
        x = self.conv3(x)
        print('after conv3',x.shape)
        #x = layers.Flatten()(x)
        print('after flatten',x.shape)
        x =  self.dense2(x)
        print(x.shape)
        x =  self.dense3(x)
        print(x.shape)
        return x
        

In [35]:
model = CustomConvModel()

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
              metrics=['accuracy'],
              experimental_run_tf_function=False
             )

NameError: name 'model' is not defined

In [ ]:
model.fit(x=x_train,
          y=y_train,
          batch_size=64,
          epochs=20,
          validation_data=(x_test, y_test),
         )

In [ ]:
for layer in model.get_weights():
    print(layer.shape)

In [ ]:
len(model.get_weights())

In [10]:
def prune_conv_layers(pruning_ratio):
    layer_to_prune = [0, 3, 6]
    pruned_weights = model.get_weights()
    
    for layer in layer_to_prune:
        converted_weights = convert_from_hwio_to_iohw(model.get_weights()[layer])
        converted_mask = convert_from_hwio_to_iohw(model.get_weights()[layer + 2]).numpy()
        for input_index, input_layer in enumerate(converted_weights):

            for kernel_index, kernel in enumerate(input_layer):
                dims = kernel.shape
                flat_weights = kernel.numpy().flatten()
                flat_masks = converted_mask[input_index][kernel_index].flatten()
                flat_weights_df = pd.DataFrame(flat_weights)
                flat_mask_df = pd.DataFrame(flat_masks)
                no_of_weights_to_prune = int(len(flat_weights)*pruning_ratio)
                #print(no_of_weights_to_prune)
                indices_to_delete = flat_weights_df.abs().values.argsort(0)[:no_of_weights_to_prune]
                for idx_to_delete in indices_to_delete:
                    flat_masks[idx_to_delete] = 0

                converted_mask[input_index][kernel_index] = flat_masks.reshape(dims)
        back_converted_mask = convert_from_iohw_to_hwio(converted_mask)
        pruned_weights[layer+2] = back_converted_mask
    
    return pruned_weights
            
            

In [12]:
pruning_ratios = [0.0, .5, 0.8, 0.9]
pre_pruning_weight_archive = []
post_pruning_weight_archive = []
post_fine_tune_weight_archive = []
pre_fine_tune_results = []
post_fine_tune_results = []

for pruning_ratio in tqdm(pruning_ratios):
    pre_pruning_weight_archive.append(model.get_weights())
    pruned_weights = prune_conv_layers(pruning_ratio)
    model.set_weights(pruned_weights)
    pruned_weights = prune_weights(model, pruning_ratio)
    model.set_weights(pruned_weights)
    post_pruning_weight_archive.append(model.get_weights())
    pre_fine_tune_results.append(model.evaluate(x_test, y_test, verbose=0))
    model.fit(x=x_train,
          y=y_train,
          batch_size=64,
          epochs=5,
          validation_data=(x_test, y_test),
         )
    post_fine_tune_results.append(model.evaluate(x_test, y_test, verbose=0))
    post_fine_tune_weight_archive.append(model.get_weights())

  0%|          | 0/4 [00:00<?, ?it/s]

(120, 84)
(84, 10)
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 62s 1ms/sample - loss: 1.6163 - accuracy: 0.8442 - val_loss: 1.6095 - val_accuracy: 0.8517
Epoch 2/5
60000/60000 [==============================] - 62s 1ms/sample - loss: 1.6157 - accuracy: 0.8454 - val_loss: 1.6094 - val_accuracy: 0.8518
Epoch 3/5
60000/60000 [==============================] - 63s 1ms/sample - loss: 1.6116 - accuracy: 0.8490 - val_loss: 1.6098 - val_accuracy: 0.8514
Epoch 4/5
60000/60000 [==============================] - 63s 1ms/sample - loss: 1.6104 - accuracy: 0.8505 - val_loss: 1.6077 - val_accuracy: 0.8533
Epoch 5/5
60000/60000 [==============================] - 62s 1ms/sample - loss: 1.6092 - accuracy: 0.8516 - val_loss: 1.6081 - val_accuracy: 0.8531


 25%|██▌       | 1/4 [05:25<16:17, 325.69s/it]

(120, 84)
(84, 10)
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 62s 1ms/sample - loss: 1.6300 - accuracy: 0.8311 - val_loss: 1.6132 - val_accuracy: 0.8475
Epoch 2/5
60000/60000 [==============================] - 63s 1ms/sample - loss: 1.6160 - accuracy: 0.8448 - val_loss: 1.6126 - val_accuracy: 0.8486
Epoch 3/5
60000/60000 [==============================] - 62s 1ms/sample - loss: 1.6110 - accuracy: 0.8498 - val_loss: 1.6087 - val_accuracy: 0.8512
Epoch 4/5
60000/60000 [==============================] - 61s 1ms/sample - loss: 1.6089 - accuracy: 0.8519 - val_loss: 1.6072 - val_accuracy: 0.8536
Epoch 5/5
60000/60000 [==============================] - 61s 1ms/sample - loss: 1.6080 - accuracy: 0.8528 - val_loss: 1.6045 - val_accuracy: 0.8556


 50%|█████     | 2/4 [10:47<10:49, 324.63s/it]

(120, 84)
(84, 10)
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 62s 1ms/sample - loss: 1.8082 - accuracy: 0.6531 - val_loss: 1.7180 - val_accuracy: 0.7448
Epoch 2/5
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.6805 - accuracy: 0.7834 - val_loss: 1.6308 - val_accuracy: 0.8329
Epoch 3/5
60000/60000 [==============================] - 67s 1ms/sample - loss: 1.6234 - accuracy: 0.8409 - val_loss: 1.6021 - val_accuracy: 0.8624
Epoch 4/5
60000/60000 [==============================] - 66s 1ms/sample - loss: 1.6006 - accuracy: 0.8635 - val_loss: 1.5888 - val_accuracy: 0.8733
Epoch 5/5
60000/60000 [==============================] - 66s 1ms/sample - loss: 1.5846 - accuracy: 0.8795 - val_loss: 1.5781 - val_accuracy: 0.8845


 75%|███████▌  | 3/4 [16:33<05:30, 330.87s/it]

(120, 84)
(84, 10)
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 62s 1ms/sample - loss: 1.8687 - accuracy: 0.5963 - val_loss: 1.8082 - val_accuracy: 0.6562
Epoch 2/5
60000/60000 [==============================] - 63s 1ms/sample - loss: 1.7732 - accuracy: 0.6915 - val_loss: 1.7118 - val_accuracy: 0.7539
Epoch 3/5
60000/60000 [==============================] - 66s 1ms/sample - loss: 1.6957 - accuracy: 0.7697 - val_loss: 1.6756 - val_accuracy: 0.7874
Epoch 4/5
60000/60000 [==============================] - 64s 1ms/sample - loss: 1.6445 - accuracy: 0.8225 - val_loss: 1.6195 - val_accuracy: 0.8480
Epoch 5/5
60000/60000 [==============================] - 61s 1ms/sample - loss: 1.6139 - accuracy: 0.8529 - val_loss: 1.6015 - val_accuracy: 0.8653


100%|██████████| 4/4 [22:03<00:00, 330.94s/it]


In [ ]:
pruned_weights

In [ ]:
model.set_weights(pruned_weights)


In [9]:
def prune_weights(model, pruning_ratio):
    weights = model.get_weights()
    weights_to_prune = model.get_weights()
    for index, weight in enumerate(weights):
        if (index == 9) or (index == 12) :
            print(weight.shape)
            flat_weights = weight.flatten()
            flat_weights_to_prune = weights_to_prune[index+2].flatten()
            #print (flat_weights_to_prune.shape, flat_weights.shape)
            flat_weights_df = pd.DataFrame(flat_weights)
            flat_weights_to_prune_df = pd.DataFrame(flat_weights_to_prune)
            no_of_weights_to_prune = int(len(flat_weights)*pruning_ratio)
            #print(no_of_weights_to_prune)
            indices_to_delete = flat_weights_df.abs().values.argsort(0)[:no_of_weights_to_prune]
            for idx_to_delete in indices_to_delete:
                flat_weights_to_prune[idx_to_delete] = 0
            dims = weights_to_prune[index+2].shape
            weights_reshaped = flat_weights_to_prune.reshape(dims)
            weights_to_prune[index+2] = weights_reshaped
    #print(weights_to_prune)
    return weights_to_prune

In [ ]:
model.evaluate(x_test, y_test)

In [14]:

for index,res in enumerate(pruning_ratios):
    print('pruning ratio: ', 
          res, 
          'accuracy before fine tuning: ',
          pre_fine_tune_results[index][1],
          'accuracy after fine tuning: ',
          post_fine_tune_results[index][1])

pruning ratio:  0.0 accuracy before fine tuning:  0.8456 accuracy after fine tuning:  0.8531
pruning ratio:  0.5 accuracy before fine tuning:  0.7877 accuracy after fine tuning:  0.8556
pruning ratio:  0.8 accuracy before fine tuning:  0.3247 accuracy after fine tuning:  0.8845
pruning ratio:  0.9 accuracy before fine tuning:  0.3505 accuracy after fine tuning:  0.8653


In [ ]:
model.get_weights()[0].shape

In [ ]:
print(convert_from_hwio_to_iohw(model.get_weights()[0]).shape)

In [8]:
def convert_from_hwio_to_iohw(weights_nchw):
    return tf.transpose(weights_nchw, [2, 3, 0, 1])



def convert_from_iohw_to_hwio(weights_nhwc):
    return tf.transpose(weights_nhwc, [2, 3, 0, 1])

In [ ]:
keras_model = tf.keras.Sequential()
keras_model.add(layers.Reshape((28, 28,1)))
#differences convolutional layer: 
#keras.layers.Conv2D uses 'glorot_uniform' initializer
#keras.layers.Conv2D uses padding method 'valid' as default
keras_model.add(layers.Conv2D(filters=6, kernel_size=(5,5), activation='tanh', padding='SAME',input_shape=(28,28,1)))
#differences pooling layer: 
#keras.layers.AvgPooling2D uses padding method 'valid' (vs. 'SAME' in custom) as default
keras_model.add(layers.AvgPool2D(padding='SAME'))
keras_model.add(layers.Conv2D(filters=16, kernel_size=(5,5), activation='tanh'))
keras_model.add(layers.AvgPool2D())
keras_model.add(layers.Flatten())
keras_model.add(layers.Dense(120, activation='tanh'))
keras_model.add(layers.Dense(84, activation='tanh'))
keras_model.add(layers.Dense(10, activation='softmax'))

In [ ]:
keras_model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                    metrics=['accuracy'],
                    experimental_run_tf_function=False
                   )

In [ ]:
keras_model.fit(x=x_train,
                y=y_train,
                batch_size=64,
                epochs=2,
                validation_data=(x_test, y_test),
         )

In [ ]:
keras_model.summary()

In [ ]:
preds = map (lambda pred: np.argmax(pred), model.predict(x_test[:30]))
preds = list(preds)

In [ ]:


plt.figure(figsize=(10,10))
for i in range(25):
    img = tf.reshape(x_test[i], shape=[28, 28])
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(img)

    plt.xlabel(preds[i])
plt.show()

In [ ]:
def weight_mask_variable(var, scope):

    mask = tf.Variable(initial_value = tf.ones(var.shape),
                       trainable=False
                      )
    return mask

def apply_mask(x, scope=''):
    #print(x.shape)
    mask = weight_mask_variable(x, scope)
    #threshold = weight_threshold_variable(x, scope)

    x = tf.cast(x, tf.float32)
    masked_weights = tf.multiply(mask, x)
    

    return masked_weights